In [ ]:
import sys
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import allel
from sklearn import metrics

In [ ]:
#####GET ADJACENT WINDOW RESULTS FROM INFERENCE RESULTS. INPUT FILES ARE OUTPUTS FROM H12_table_for_ROC_snps.py AND
#####SF_table_for_ROC_wins.py#####
winSize=10000
regionLen=997204
#Set col names for categorising FPs, TPs etc (adjust for SweepFinder2 analysis)
col1  = 'H12'
col2 = 'sweep'
choices = ['FN','TP','FP','TN']

#Loop through 2Nes values
for s in [100, 1000, 10000]:
    #Create df to store all inference results
    rdf = pd.DataFrame()
    #list to store inferenc results
    df_lst = []
    #Loop through replicates
    for i in range(1, 201):
        try:
            #Read in inference file and concatenate to rdf
            df = pd.read_csv(inferenceFile, sep='\t', header=0)
            df.columns = [col2, col1]
            rdf = pd.concat([rdf, df])
            #Append df to list
            df_lst.append(df)
        except Exception:
            pass

    rdf = rdf.dropna()
    #Calculate ROC statistics
    fpr, tpr, thresholds = metrics.roc_curve(rdf.sweep, rdf['H12'])

    res = []
    #Set number of thresholds for adjacent windows method using min and max thresholds from metrics.roc_curve function above
    new_thresholds = np.linspace(np.min(thresholds), np.max(thresholds), 1000)
    with open(outFile, 'a') as outfile:
        #Loop through thresholds
        for i, t in enumerate(new_thresholds):
            rdf = pd.DataFrame()
            TP = 0
            TN = 0
            FP = 0
            FN = 0
            #Loop through dfs (ie replicates)
            for df in df_lst:
                #Set conditions using thresholds and apply to column
                conditions = [(df[col1]<t) & (df[col2]==1), 
                             (df[col1]>=t) & (df[col2]==1),
                             (df[col1]>=t) & (df[col2]==0),
                             (df[col1]<t) & (df[col2]==0)]

                df['outcome'] = np.select(conditions, choices, default=np.nan)

                lst = list(df.outcome)
                #Set column which will be the final results column
                df['final_outcome'] = df.outcome
                #Set columns shifted one forward and back (ie adjacent windows)
                df["next_outcome"] = df.outcome.shift(-1) 
                df["prior_outcome"] = df.outcome.shift(1)
                #While loop keeps until there are no instances where a true positive is next to a false positive
                while(len(df[(df.next_outcome.eq("TP") & df.final_outcome.eq("FP")) |
                   (df.prior_outcome.eq("TP") & df.final_outcome.eq("FP"))]) > 0):
                    #If the next window (either before or after) is a TP, set to TP
                    df['final_outcome'] = np.where(df.next_outcome.eq("TP") & df.outcome.eq("FP") & (df[col1]>=t), 
                                                   'TP', df.final_outcome)
                    df['final_outcome'] = np.where(df.prior_outcome.eq("TP") & df.outcome.eq("FP") & (df[col1]>=t), 
                                                   'TP', df.final_outcome)

                rdf = pd.concat([rdf, df])
            #Get the value counts for each category and output to file
            if('TP' in rdf.final_outcome.value_counts()):
                TP += rdf.final_outcome.value_counts()['TP']
            if('FP' in rdf.final_outcome.value_counts()):
                FP += rdf.final_outcome.value_counts()['FP']
            if('TN' in rdf.final_outcome.value_counts()):
                TN += rdf.final_outcome.value_counts()['TN']
            if('FN' in rdf.final_outcome.value_counts()):
                FN += rdf.final_outcome.value_counts()['FN']
            res.append([TP,FP,TN,FN])
            outfile.write('\t'.join([str(x) for x in [t,TP,FP,TN,FN]]) + '\n')